# IRIDE

$\theta$ = 90°

iride aperta -> $\nu \simeq$ 230kHz

iride media -> $\nu \simeq$ = 120kHz

iride chiusa -> $\nu \simeq$ = 45kHz

nota, i valori sono approssimativi $\pm$ 20kHz 

$\nu$ = frequenza fotoni rilevati

In [8]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val

In [50]:
# Constants
n = 1.33 # index of refraction
η = 10**-9  # g/(nm*s) viscosity of water
λ = 633  # nm wavelength of the laser
T = 293  # K temperature
KB = 1.3806e-2  # g (nm)^2 / s^2 K boltzmann constant

In [40]:
iride_chiusa = pd.read_csv(dir_path + '\\data\\g_90_45kHz.txt',sep='\t ',header=None,engine='python')
iride_chiusa = iride_chiusa[(iride_chiusa[0] >= 0.1) & (iride_chiusa[0] <= 10)]
iride_aperta = pd.read_csv(dir_path + '\\data\\g_90_230kHz.txt',sep='\t ',header=None,engine='python')
iride_aperta = iride_aperta[(iride_aperta[0] >= 0.1) & (iride_aperta[0] <= 10)]
iride_media = pd.read_csv(dir_path + '\\data\\g_90.txt',sep='\t ',header=None,engine='python')
iride_media = iride_media[(iride_media[0] >= 0.1) & (iride_media[0] <= 10)]

θ = 90
sigmay = np.float64(0.001)

# iride_chiusa.head()

In [49]:
f = lambda t,A,B,γ: A*np.exp(-γ*t) + B
iride_chiusa.head()
fit_chiusa = Interpolazione(iride_chiusa[0].to_numpy()/1000, iride_chiusa[1].to_numpy(),sigmay,f,[0.01,0.0001,2000],names=['A','B','γ'])
fit_aperta = Interpolazione(iride_aperta[0].to_numpy()/1000,iride_aperta[1].to_numpy(),sigmay,f,[0.01,0.0001,2000],names=['A','B','γ'])
fit_media = Interpolazione(iride_media[0].to_numpy()/1000,iride_media[1].to_numpy(),sigmay,f,[0.01,0.0001,2000],names=['A','B','γ'])
print('-----------------Chiusa------------------')
print(fit_chiusa)
print('-----------------Aperta------------------')
print(fit_aperta)
print('-----------------Media------------------')
print(fit_media)

gamma_aperta,sigma_gamma_aperta = fit_aperta.values['γ'],fit_aperta.errors['γ']
gamma_chiusa,sigma_gamma_chiusa = fit_chiusa.values['γ'],fit_chiusa.errors['γ']
gamma_media,sigma_gamma_media = fit_media.values['γ'],fit_media.errors['γ']

-----------------Chiusa------------------
----------------- VALORI FIT: -----------------
A: (1.026 ± 0.008)e-1 
B: (1.476 ± 2.077)e-4 
γ: (2.712 ± 0.038)e3 

dof: 50
chi2: 18.93
chi2 ridotto: 0.38
pvalue: 1.0
------------------------------------------------

-----------------Aperta------------------
----------------- VALORI FIT: -----------------
A: (2.806 ± 0.08)e-2 
B: (1.035 ± 2.097)e-4 
γ: (2.626 ± 0.133)e3 

dof: 50
chi2: 1.31
chi2 ridotto: 0.03
pvalue: 1.0
------------------------------------------------

-----------------Media------------------
----------------- VALORI FIT: -----------------
A: (4.93 ± 0.08)e-2 
B: (1.077 ± 2.096)e-4 
γ: (2.634 ± 0.076)e3 

dof: 50
chi2: 3.33
chi2 ridotto: 0.07
pvalue: 1.0
------------------------------------------------



In [48]:
x,y = iride_chiusa[0]/1000, iride_chiusa[1]

color_chiusa = px.colors.sequential.Plasma[1]
color_media = px.colors.sequential.Plasma[3]
color_aperta = px.colors.sequential.Plasma[5]

fig = go.Figure()
fig.add_trace(go.Scatter(x=iride_chiusa[0]/1000,y=iride_chiusa[1], mode='markers', name=f' 45Khz',
        marker=dict(color = color_chiusa),
        error_y=dict( 
			type='constant', 
			value=sigmay, 
			color=color_chiusa,
			thickness=1.5, 
			width=3, 
		)))

fig.add_trace(go.Scatter(x=iride_aperta[0]/1000,y=iride_aperta[1], mode='markers', name=f'230kHz',
        marker=dict(color = color_aperta),
        error_y=dict( 
            type='constant', 
            value=sigmay, 
            color=color_aperta,
            thickness=1.5, 
            width=3, 
        )))

fig.add_trace(go.Scatter(x=iride_media[0]/1000,y=iride_media[1], mode='markers', name=f'130kHz',
        marker=dict(color = color_media),
        error_y=dict( 
            type='constant', 
            value=sigmay, 
            color=color_media,
            thickness=1.5, 
            width=3, 
        )))

X,Y = fit_chiusa.draw()
fig.add_trace(go.Scatter(x=X,y=Y, mode='lines',line_color=color_chiusa, showlegend=False,hoverinfo='skip'))

X,Y = fit_aperta.draw()
fig.add_trace(go.Scatter(x=X,y=Y, mode='lines',line_color=color_aperta, showlegend=False,hoverinfo='skip'))

X,Y = fit_media.draw()
fig.add_trace(go.Scatter(x=X,y=Y, mode='lines',line_color=color_media, showlegend=False,hoverinfo='skip'))


fig.update_layout(
        xaxis_type="log",
        yaxis_title="g<sup>2</sup>(τ)",
        xaxis_title="t (s)",
        # title='g<sup>2</sup>(τ) vs t',
        xaxis=dict(range=[np.log10(0.0001), np.log10(0.010)]),
        width=800,
        height=600,
        title={
            'text': "Correlation Function g<sup>2</sup>(τ)",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font':dict(size=30)
        },
        plot_bgcolor='white',
        font=dict(
            #family="Courier New, monospace",
            size=18,
            color="Black"
        ),
            # yaxis=dict(range=[0, 0.06])
    )

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    range=[0, 0.1],
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.show()

In [51]:
def calculate_R(gamma, theta):
    theta_rad = np.radians(theta)
    R = (16 * np.pi * n**2 * T * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma)
    return R

def calculate_sigma_R(gamma, sigma_gamma, theta):
    theta_rad = np.radians(theta)
    sigma_R = (16 * np.pi * n**2 * T * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma**2) * sigma_gamma
    return sigma_R


R_chiusa = calculate_R(gamma_chiusa, θ)
sigma_R_chiusa = calculate_sigma_R(gamma_chiusa, sigma_gamma_chiusa, θ)

R_aperta = calculate_R(gamma_aperta, θ)
sigma_R_aperta = calculate_sigma_R(gamma_aperta, sigma_gamma_aperta, θ)

R_media = calculate_R(gamma_media, θ)
sigma_R_media = calculate_sigma_R(gamma_media, sigma_gamma_media, θ)

print('R_chiusa: ',final_val(R_chiusa,sigma_R_chiusa,2,0,'nm'))
print('R_aperta: ',final_val(R_aperta,sigma_R_aperta,2,0,'nm'))
print('R_media: ',final_val(R_media,sigma_R_media,2,0,'nm'))

R_chiusa:  55.17 ± 0.77 nm
R_aperta:  56.97 ± 2.9 nm
R_media:  56.79 ± 1.65 nm
